In [1]:
# https://pypi.org/project/torch-geometric/
# pip install torch
# pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-geometric


# https://towardsdatascience.com/louvains-algorithm-for-community-detection-in-python-95ff7f675306
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter 

In [2]:
# plot the data
from utils.visual import plot

#plot(0) 

In [3]:
import re

def find_train_ego_index():
    pre, pree = [], []
    for item in train.files:
        pre.append(int(re.sub('\D', '',item )))

    for i in egonet.files:
        pree.append(int(re.sub('\D', '',i )))
        
    train_ego_index = []
    for i in range(len(pree)):
        if pree[i] in pre:
            train_ego_index.append(i)
    return train_ego_index, pree, pre

In [4]:
# self-made packages
from utils.data import data_preprocess


train = data_preprocess("\Training")
train_list, train_dict = train.train_data()
df = pd.DataFrame(data=train_dict)

In [5]:
egonet = data_preprocess("\egonets")
ego_list, user_list = egonet.ego_data()

In [ ]:
# 這個是指應該將哪些 ego_list 用在 training，方便輸入0,1,2，就不用輸入 239、345、661...
train_ego_index,ego_user, train_user  = find_train_ego_index()

## Data Preparation

In [ ]:
from utils.user import users

users = users()
index = user_list[train_ego_index[2]] # 更改數字可以挑選要哪一個 training data 

# edge information EX: (240, 241), (240, 242)...
edge1 = egonet.edge_data(train_ego_index[0])

In [ ]:
#users = pd.read_csv('df_new_friend.csv') #把註解拿掉代表讀取新的feature

## Predict Data

In [ ]:
circlesP = {}

# 這裡是整理所有的 predict data，但因為目前沒有所以暫時用 training 的代替
for i in range(len(train_dict)):
    uidP,friendsP = train_dict[i]['UserId'], train_dict[i]['Predicted'] # predict 
    circlesP[int(uidP)] = [set([int(x) for x in c.split()]) for c in friendsP.split(';')]

## Connected Component
#### 測試connected component

In [ ]:
from connected_components import findCommunities
import sys, os

def connected_componet(target_dir):
    egoUser = -1
    try:
        egoUser = int(re.sub("\D", '', target_dir))
    except Exception as e:
        print("Expected files to be names 'X.egonet' where X is a user ID")

    cs = list(findCommunities(target_dir))

    if len(cs) == 0:
        cs = [set(adj.keys())]
    cs = [' '.join([str(y) for y in x]) for x in cs]
    res = str(egoUser) + ',' + ';'.join(cs)
    
    return res
    

In [ ]:
def process(test_ego_num):
    return [{
            'UserId': test_ego_num.split(',')[0],
            'Predicted': test_ego_num.split(',')[1]
            }]


In [ ]:
connect_egonet = []
for i in range(len(train.files)):
    target_dir = os.getcwd()+"\\egonets\\"+train.files[i].replace('circles', 'egonet')
    test_ego_num = connected_componet(target_dir)
    test = process(test_ego_num)
    connect_egonet+=(test)

# Caculate Loss
### 之後可能要放進一個mode裡面當作 objective function 之類的，這邊先暫時表達一次性的評分

## Model

In [ ]:
from sklearn.cluster import KMeans

def exe(n_clusters):
    km = KMeans(
        n_clusters=n_clusters, init='random',
        n_init=10, max_iter=300, 
        tol=1e-04, random_state=2
    )
    final, indexx = [], []
    for i in range(len(train_ego_index)):
        index = user_list[train_ego_index[i]] # 更改數字可以挑選要哪一個 training data 
        test = users.iloc[index]

        X = np.array(test).astype(np.int64)
        y_km = km.fit(X)
        predict = y_km.predict(X)
        final.append(predict)
        indexx.append(index)
    return final, indexx

In [ ]:
from socialCircles_metric import loss1

def temperal(train_dict):
        
    circlesG = {}

    for i in range(len(train_dict)):
        uidG,friendsG = train_dict[i]['UserId'], train_dict[i]['Predicted'] # ground truth
        circlesG[int(uidG)] = [set([int(x) for x in c.split()]) for c in friendsG.split(';')] 
        
        
    totalLoss = 0
    for k in circlesP.keys():
        if k not in circlesG:
            print("Ground-truth has prediction for circle", k, "but prediction does not")
            break

        l = loss1(circlesG[k], circlesP[k])
        #print("loss for user", k, "=", l)
        totalLoss += l
    print("total loss for all users =", totalLoss)

In [ ]:
def data(n_clusters):

    final, index = exe(n_clusters)
    
    resss = []
    for k in range(len(final)):
        fina = ""
        for i in range(n_clusters):
            tem = ''

            for j in range(len(final[k])):
                if final[k][j] == i:
                    tem+= ' ' + str(index[k][j])
            fina += tem.strip() + ';'
        resss.append(fina)
        
    transform_label = []
    
    for i in range(len(index)):
        transform_label.append({"UserId":index[i][0]-1, "Predicted":resss[i].strip()[:-1]})
        
    return transform_label

In [ ]:
temperal(connect_egonet)

In [ ]:
temperal(train_dict)

In [6]:
ego_list[0]

[[1,
  146,
  189,
  229,
  201,
  204,
  60,
  215,
  35,
  91,
  238,
  88,
  28,
  166,
  218,
  156,
  6,
  2,
  8,
  231,
  165,
  184,
  20,
  221,
  128,
  39,
  48,
  109,
  84,
  15,
  139,
  21,
  108,
  207,
  161,
  147,
  155,
  140,
  41,
  12,
  160,
  127,
  227,
  47,
  65,
  105,
  62,
  182,
  79,
  90,
  100,
  148,
  137,
  170,
  158,
  58,
  134,
  179,
  123,
  167,
  45,
  51,
  85,
  54,
  75],
 [2,
  146,
  191,
  229,
  201,
  204,
  60,
  215,
  35,
  95,
  91,
  141,
  235,
  88,
  166,
  218,
  156,
  6,
  231,
  165,
  184,
  1,
  221,
  46,
  128,
  210,
  39,
  48,
  109,
  84,
  15,
  139,
  21,
  108,
  207,
  161,
  147,
  155,
  140,
  126,
  12,
  160,
  127,
  227,
  47,
  129,
  65,
  205,
  105,
  62,
  79,
  90,
  100,
  148,
  137,
  170,
  76,
  97,
  58,
  134,
  179,
  167,
  45,
  51,
  85,
  54,
  75],
 [3,
  185,
  80,
  61,
  188,
  22,
  222,
  118,
  4,
  223,
  138,
  52,
  212,
  213,
  226,
  117,
  103,
  18,
  154,
  43,
  176,
